# **Project 1 - Search-based solutions for static Pac-Man game**
**Subject:** MC906/MO416 - Introduction to Artificial Intelligence 

**Authors:**

    Daniel Helú Prestes de Oliveira - RA 166215
    Eduardo Barros Innarelli        - RA 170161
    Matheus Rotta Alves             - RA 184403
    Victor Ferreira Ferrari         - RA 187890
    Vinícius Couto Espindola        - RA 188115


## **Problem**

> TODO List:
- [ ] Problem description 
- [ ] Problem modeling
- [ ] Search agent (motivation, API)

### **Description**
Consider the Pacman video game. We're given a maze where each position can have one of the following: start (unique), goal (unique), enemy, dots, walls or empty spaces. The pacman will begin the game in the start position and will try to reached the goal position only crossing through empty spaces and dots positions. In the event were pacman reaches outside the map, it's position is "wrapped around" the map, meaning that it will appear on the oposite side of the map maintaining one of it's coodinates.
When compared to the original game, some simplifications were made: 
 - Ghosts do not move during the pacmans search (their initial position is their only position) 
 - Berries were removed from the game, therefore Pacman has no countermeasure against ghosts.

### **Modeling**
 For every move pacman makes, it will pay a cost of 1 point, however, if a dot is found, it will receive a payment of 10 points. The sum of all costs and payments will be considered the score achieve by Pacman when executing a certain path. Also, Pacman can only move one position at a time. 
Based on such maze and restrictions, we must seek the best feasible path for Pacman to reach the goal. In our case, "best" refers to the path in which we achieve the highest score.

**Environment:** Is the maze described by the problem.

**Actuator:** The only action allowed to pacman is to move to a neighbor cel in the maze given the problems restrictions.

**Sensors:** We consider that pacman can "see" the entirety of the given maze.

**Known:** The consquences to the environment given an action are completely predictable.

**Environment Properties:**
 - **Fully observable:** We can see the entire maze.
 - **Deterministic:** The next state is entirely defined by the current state an a possible action on it.
 - **Static:** The only change that occurs within the maze is cause by Pacman himself, the ghosts do not act.
 - **Single Agent:** The only agent within the problem is Pacman.
 - **Observability:** ??
 - **Sequential:** The state of the environment is completely predictable and is only affected by Pacman himself.

## **Test Cases**

> TODO List:
- [ ] Maybe show two examples (one dense and it's correspondent sparse)

For testing purposes, we generated 10 mazes using the [tool provided by classmate Gabriel Bomfim](https://gabomfim.github.io/pacman-mazegen/tetris/many.htm) in Google Classrom, which adapts the [maze generator](https://shaunlebron.github.io/pacman-mazegen/) linked in the project description. Each tile is represented by a char, where **|** and **-** are walls, **.** are foods and **o** are ghosts. For each maze, we choosed three start and goal positions, respectively symbolized by **!** and **?**.

As this tool creates mazes fully filled with food, we thought that it would be good for comparision to also test sparse mazes, which we created by randomly removing dots in the dense ones. These variations, together with the originals, give us a total of 60 mazes, stored in `./mazes` directory.

In [1]:
from mazes.get_mazes import get_mazes
denses, sparses = get_mazes()

## **Uninformed Search Methods**

> TODO List:
- [X] Short theoretical introduction

The Uninformed Search Methods, also known as Blind Search Methods, are algorithms that are given no information about the problem other than its definition. They are only able to generate possible successor of a state and analyze this sucessors in a sequence according to the algorithm applied. They analyze each successor created until it finds the one state that is equal to the goal state.

### **Breadth-First Search Solution**
###### **Responsible:** Victor

> TODO List:
- [ ] Short theoretical introduction
- [ ] Run tests script with and without maze in state
- [ ] Results table 
- [ ] Analysis with relevant(s) animation(s)

### **Depth-First Search Solution**
###### **Responsible:** Daniel

> TODO List:
- [X] Short theoretical introduction
- [ ] Run tests script with and without maze in state
- [ ] Results table 
- [ ] Analysis with relevant(s) animation(s)

The Depth-First Search is a Uninformed Search method that expands its deepest node possible first, the one that has no sucessor. If the node is the goal state, it has found the path to the solution, otherwise it goes back to the most recent ancestor node that still has successor that were not expanded and it expands the deepest sucessor this ancestor node has.

For the Pacman Problem the goal state considered for the problem was the final position that pacman should go. It did not consider analyzing all the possible outcomes of eating the dots to increase it points because the time and memory to consider all possible states would be exponential.

## **Informed Search Methods**

> TODO List:
- [ ] Short theoretical introduction

### **A* Search Solution**
###### **Responsible:** Eduardo

> TODO List:
- [ ] Run tests script with and without maze in state
- [ ] Results table
- [ ] Analysis with relevant(s) animation(s)

As an informed search algorithm, A\* takes into account information about the path cost together with an heuristic to evaluate which is the most promising path to take when it enters a state. For this evaluation, A\* chooses in state $n$ to proceed to the neighbor that gives the lowest $f(n) = g(n) + h(n)$, $g(n)$ being the exact path cost from starting state to $n$ and $h(n)$ the heuristic estimated cost from $n$ to goal state.

#### **Heuristic**

How fast the agent reaches the goal in A\* depends on the heuristic implemented and how it affects the nodes expansion. Without the maze in state this isn't that much of a concern, as for a $n\times m$ maze there can be up to $O(nm)$ states. It's pretty hard, though, to estimate a good cost to goal without the current configuration knowledge (the combinations of foods can make any estimation over the initial maze very far from optimal). To simplify, we use the **Manhattan distance** as a heuristic for this variation of the problem, which is the distance between the agent and the goal positions measured along axes at right angles (i.e., $|x_1 - x_2| + |y_1 - y_2|$, given that the agent is in $(x_1, y_1)$ and the goal is to reach $(x_2, y_2)$). It's a admissible heuristic as there can't be a shorten path from node to goal.

With maze in state it's specially important to pick a good heuristic - after all, the search space is exponentially large, as each subset of eaten food represents a different node even with the agent in a fixed position. Considering that in this case we have information that allows a more realistic approach, but keeping it simple in terms of code, we implement the sum of Manhattan distances between the agent and all the foods as a heuristic, as it's highly possible that most of them will be eaten in the optimal path. Notice that this sum can overestimate the optimal, because it's not always true that all the foods will be eaten in the best path. As a overestimating heuristic, it breaks admissibility - that is, A\* is not guaranteed to find the optimal path. Even so, as our problem gives a high score to Pac-Man when it eats, we chose it expecting A\* will find good paths in reasonable running times. 


In [2]:
# global reference to goal => problem 1 heuristic needs it
# TODO - Avoid this (hardcode)
goal_ref = None

# Heuristic for problem 1 - without maze in state
def astar_heuristic_p1(node):
    ''' manhattan distance between Pac-Man and goal '''
    
    idx = node.state
    md = manhattan_distance(goal_ref, idx)
    return md

# Heuristic for problem 2 - with maze in state
def astar_heuristic_p2(node):
    ''' sum of manhattan distances between Pac-Man and all foods in maze '''
    
    # Detach maze configuration and Pac-Man position
    tuple_maze, idx = node.state
    
    # Accumulate sum of manhattan distances to foods
    md_sum = 0
    for food_idx in np.argwhere(maze == '.'):
        md_sum += manhattan_distance(food_idx, idx)
            
    return md_sum

### **? Search Solution**
###### **Responsible:** Matheus

> TODO List:
- [ ] Short theoretical introduction
- [ ] Heuristics choosen for each problem variation
- [ ] Run tests script with and without maze in state
- [ ] Results table
- [ ] Analysis with relevant(s) animation(s)

## **Local Search Methods**

> TODO List:
- [ ] Short theoretical introduction

### **Simulated Annealing Solution**
###### **Responsible:** Vinicius

> TODO List:
- [ ] Short theoretical introduction
- [ ] Heuristics choosen for each problem variation
- [ ] Run tests script with and without maze in state
- [ ] Results table
- [ ] Analysis with relevant(s) animation(s)

## **Comparisions**

> TODO List:
- [ ] Compare methods and problems
- [ ] Maybe display a graph comparing scores
- [ ] Maybe animate one maze with all methods running (different colors to distinguish agents)